In [1]:
#  imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array

In [2]:
# load data
import names
df = pd.read_csv("htru.csv", names = names.names)

In [3]:
# head of data (5)
df.head()

,Profile_mean,Profile_stdev,Profile_skewness,Profile_kurtosis,DM_mean,DM_stdev,DM_skewness,DM_kurtosis,class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0


In [4]:
# df to values
df = df.values

In [5]:
# Deep RandomForest
import argparse
import numpy as np
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/htru.json")  
gc = GCForest(config)

In [7]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,8]
X = df[:,0:8]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [8]:
# count class
len(np.unique(y_train))

2

In [9]:
     # X_enc is the concatenated predict_proba result of RandomForest

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
     

[ 2018-04-21 17:17:37,303][cascade_classifier.fit_transform] X_groups_train.shape=[(12528, 8)],y_train.shape=(12528,),X_groups_test.shape=[(5370, 8)],y_test.shape=(5370,)
[ 2018-04-21 17:17:37,306][cascade_classifier.fit_transform] group_dims=[8]
[ 2018-04-21 17:17:37,307][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-21 17:17:37,309][cascade_classifier.fit_transform] group_ends=[8]
[ 2018-04-21 17:17:37,311][cascade_classifier.fit_transform] X_train.shape=(12528, 8),X_test.shape=(5370, 8)
[ 2018-04-21 17:17:37,313][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(12528, 8), X_cur_test.shape=(5370, 8)
[ 2018-04-21 17:17:38,109][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-21 17:17:39,021][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.69%
[ 2018-04-21 17:17:39,943][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_

In [10]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of RandomForest = {:.6f} %".format(acc * 100))

[ 2018-04-21 17:17:46,566][cascade_classifier.transform] X_groups_test.shape=[(5370, 8)]
[ 2018-04-21 17:17:46,568][cascade_classifier.transform] group_dims=[8]
[ 2018-04-21 17:17:46,571][cascade_classifier.transform] X_test.shape=(5370, 8)
[ 2018-04-21 17:17:46,573][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(5370, 8)


Test Accuracy of RandomForest = 98.044693 %


In [15]:
# predict_proba result of RandomForest + data
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-21 17:17:48,183][cascade_classifier.transform] X_groups_test.shape=[(5370, 8)]
[ 2018-04-21 17:17:48,185][cascade_classifier.transform] group_dims=[8]
[ 2018-04-21 17:17:48,186][cascade_classifier.transform] X_test.shape=(5370, 8)
[ 2018-04-21 17:17:48,188][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(5370, 8)


In [16]:
X_train_new = X_train_enc
X_test_new  = X_test_enc

In [17]:
X_test_new.shape

(5370, 10)

In [18]:
X_train_new.shape

(12528, 10)

In [19]:
# Cascade RandomForest
config = load_json("./examples/htrugc.json")  
gc = GCForest(config)

X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-21 17:17:49,855][cascade_classifier.fit_transform] X_groups_train.shape=[(12528, 10)],y_train.shape=(12528,),X_groups_test.shape=[(5370, 10)],y_test.shape=(5370,)
[ 2018-04-21 17:17:49,858][cascade_classifier.fit_transform] group_dims=[10]
[ 2018-04-21 17:17:49,859][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-21 17:17:49,861][cascade_classifier.fit_transform] group_ends=[10]
[ 2018-04-21 17:17:49,863][cascade_classifier.fit_transform] X_train.shape=(12528, 10),X_test.shape=(5370, 10)
[ 2018-04-21 17:17:49,865][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(12528, 10), X_cur_test.shape=(5370, 10)
[ 2018-04-21 17:17:50,610][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-21 17:17:51,509][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.69%
[ 2018-04-21 17:17:52,413][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator

[ 2018-04-21 17:18:27,276][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=97.85%
[ 2018-04-21 17:18:28,176][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=98.80%
[ 2018-04-21 17:18:29,166][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=97.37%
[ 2018-04-21 17:18:30,020][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=98.16%
[ 2018-04-21 17:18:30,935][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=97.36%
[ 2018-04-21 17:18:31,832][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=97.04%
[ 2018-04-21 17:18:32,699][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=98.08%
[ 2018-04-21 17:18:33,580][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=98.56%


In [20]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of RandomForest = {:.6f} %".format(acc * 100),i,'fois')

In [21]:
# deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    
    config = load_json("./examples/htrugc.json")  
    gc = GCForest(config)
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()    

[ 2018-04-21 17:18:52,684][cascade_classifier.fit_transform] X_groups_train.shape=[(12528, 10)],y_train.shape=(12528,),X_groups_test.shape=[(5370, 10)],y_test.shape=(5370,)
[ 2018-04-21 17:18:52,686][cascade_classifier.fit_transform] group_dims=[10]
[ 2018-04-21 17:18:52,687][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-21 17:18:52,689][cascade_classifier.fit_transform] group_ends=[10]
[ 2018-04-21 17:18:52,691][cascade_classifier.fit_transform] X_train.shape=(12528, 10),X_test.shape=(5370, 10)
[ 2018-04-21 17:18:52,693][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(12528, 10), X_cur_test.shape=(5370, 10)
[ 2018-04-21 17:18:53,451][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-21 17:18:54,422][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.69%
[ 2018-04-21 17:18:55,291][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator

[ 2018-04-21 17:19:30,289][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=97.61%
[ 2018-04-21 17:19:31,173][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=98.56%
[ 2018-04-21 17:19:32,070][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=97.69%
[ 2018-04-21 17:19:32,923][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=98.24%
[ 2018-04-21 17:19:33,871][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=97.60%
[ 2018-04-21 17:19:34,759][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=96.65%
[ 2018-04-21 17:19:35,590][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=97.92%
[ 2018-04-21 17:19:36,490][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=98.40%


('Test Accuracy of RandomForest = 98.063315 %', 0, 'fois')


[ 2018-04-21 17:19:41,392][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.09%
[ 2018-04-21 17:19:42,313][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.69%
[ 2018-04-21 17:19:43,194][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.85%
[ 2018-04-21 17:19:44,225][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.61%
[ 2018-04-21 17:19:45,118][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.00%
[ 2018-04-21 17:19:46,047][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.24%
[ 2018-04-21 17:19:46,973][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.52%
[ 2018-04-21 17:19:47,865][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.16%


[ 2018-04-21 17:20:22,846][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=97.04%
[ 2018-04-21 17:20:23,730][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=97.84%
[ 2018-04-21 17:20:24,740][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=98.48%
[ 2018-04-21 17:20:25,646][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=97.92%
[ 2018-04-21 17:20:25,778][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=97.91%
[ 2018-04-21 17:20:25,779][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=98.16%
[ 2018-04-21 17:20:25,782][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=97.91%
[ 2018-04-21 17:20:25,784][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=98.16%
[ 2018-04-21 17:20:25,787][

...

[ 2018-04-21 17:38:07,140][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.01%
[ 2018-04-21 17:38:08,041][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.85%
[ 2018-04-21 17:38:08,900][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.77%
[ 2018-04-21 17:38:09,775][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.69%
[ 2018-04-21 17:38:10,653][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.77%
[ 2018-04-21 17:38:11,526][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.16%
[ 2018-04-21 17:38:12,463][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.28%
[ 2018-04-21 17:38:13,307][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.24%


[ 2018-04-21 17:38:48,520][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=96.88%
[ 2018-04-21 17:38:49,422][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=98.16%
[ 2018-04-21 17:38:50,334][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=98.48%
[ 2018-04-21 17:38:51,176][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=97.84%
[ 2018-04-21 17:38:51,389][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=97.93%
[ 2018-04-21 17:38:51,391][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=98.06%
[ 2018-04-21 17:38:51,393][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=97.93%
[ 2018-04-21 17:38:51,395][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=98.06%
[ 2018-04-21 17:38:51,398][

('Test Accuracy of RandomForest = 98.081937 %', 18, 'fois')


[ 2018-04-21 17:39:16,314][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.93%
[ 2018-04-21 17:39:17,208][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.77%
[ 2018-04-21 17:39:18,136][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.77%
[ 2018-04-21 17:39:18,986][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.77%
[ 2018-04-21 17:39:19,904][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.77%
[ 2018-04-21 17:39:20,802][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.16%
[ 2018-04-21 17:39:21,696][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.68%
[ 2018-04-21 17:39:22,574][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.00%


('Test Accuracy of RandomForest = 97.988827 %', 19, 'fois')


# ('Test Accuracy of RandomForest = 98.268156 %', 15, 'fois')